In [ ]:
import pandas as pd
import numpy as np
import ampal
from dp_utils.pipeline_data import get_full_pipeline_df
from utils import useful_cols,useful_cols_non_csv
import pandas

pandas.set_option("display.max_rows", None, "display.max_columns", None)

In [ ]:

def generate_deltaprot_designs_data():

    no_disulfide_df =pd.read_csv("/home/tadas/code/deltaproteinsBristol/selected_deltaprots/no_disulfide/no_disulfide_selected_deltaprots.csv")
    no_disulfide_whole_pipeline_df = get_full_pipeline_df("/home/tadas/code/deltaproteinsBristol/pipeline_data/05_evaluation")
    # merge no_disulfide_whole_pipeline_df into no_disulfide_df only where by model_sequence and sequence_name match. Also ignore duplicate columns
    no_disulfide_df = pd.merge(no_disulfide_df, no_disulfide_whole_pipeline_df, how="left", left_on=["sequence_name", "model_sequence"], right_on=["sequence_name", "model_sequence"], suffixes=("", "_whole_pipeline"))
    no_disulfide_df.drop(columns=[col for col in no_disulfide_df.columns if "_whole_pipeline" in col], inplace=True)


    variable_linkers_df = pd.read_csv("/home/tadas/code/deltaproteinsBristol/selected_deltaprots/variable_linkers/variable_linkers/variable_linkers_selected_deltaprots.csv")
    variable_linkers_whole_pipeline_df = get_full_pipeline_df("/home/tadas/code/deltaproteinsBristol/pipeline_data/orientation_gap_length_optimisation_optimised")
    variable_linkers_df = pd.merge(variable_linkers_df, variable_linkers_whole_pipeline_df, how="left", left_on=["sequence_name", "model_sequence"], right_on=["sequence_name", "model_sequence"], suffixes=("", "_whole_pipeline"))
    variable_linkers_df.drop(columns=[col for col in variable_linkers_df.columns if "_whole_pipeline" in col], inplace=True)


    no_disulfide_df["name"]="no_disulfide_"+no_disulfide_df["orientation_code"]
    variable_linkers_df["name"]="variable_linkers_"+variable_linkers_df["orientation_code"]

    # merge the two piplene dataframes 
    df = pd.concat([no_disulfide_df, variable_linkers_df], axis=0)

    well_df = pd.read_csv("/home/tadas/code/deltaproteinsBristol/order_optimised_codons_96_wp.csv")

    # merge well_df with df by Name and sort by Well Position
    df = pd.merge(well_df, df, how="left", left_on="Name", right_on="name")



    # MGSSHHHHHHSSGENLYFQSGS addition adds some mw
    df["model_sequence_w_prefix"] = "MGSSHHHHHHSSGENLYFQSGS" + df["model_sequence"]
    df["mass_w_prefix"] = df["model_sequence_w_prefix"].apply(ampal.analyse_protein.sequence_molecular_weight)
    df["sequence_molar_extinction_280_w_prefix"] = df["model_sequence_w_prefix"].apply(ampal.analyse_protein.sequence_molar_extinction_280)
    
    df["percent_helicity_with_prefix"] = df["dssp_assignment"].str.count("H") / df["model_sequence_w_prefix"].str.len()
    
    df.to_pickle("/home/tadas/code/deltaproteinsBristol/deltaprot_designs_data.pkl")

def load_deltaprot_designs_data():
    return pd.read_pickle("/home/tadas/code/deltaproteinsBristol/deltaprot_designs_data.pkl")

def calc_molar_conc(ext_coef, absorbance):
    return ext_coef / absorbance

def calc_mg_per_ml(molar_conc, mass):
    return molar_conc * mass

In [ ]:
generate_deltaprot_designs_data()
df = load_deltaprot_designs_data()


In [ ]:

# populate received_from_idt column. Received everything except for C2,C4,D5,D6,F2,F3,F6


not_received = ['C2', 'C4', 'D5', 'D6', 'F2', 'F3', 'F6']
df['received_from_idt'] = ~df["Well Position"].isin(not_received)

# populate transformation_attempted for (A1-A12,B1-B12,C1,C3,F4,C5,C6)
transformation_attempted_ids = set([f"A{i}" for i in range(1, 13)] +
                    [f"B{i}" for i in range(1, 13)] +
                    ['C1', 'C3', 'F4', 'C5', 'C6'])
df['transformation_attempted'] = df["Well Position"].isin(transformation_attempted_ids)
df['transformation_successful'] = df["Well Position"].isin(transformation_attempted_ids) # all transformations worked, C6 failed initially

# populate expression_levels: (None, "low","medium","high") low: A1,A3,A6,A7,A8,A10,B1. Medium: A5,A11,B2,B3. High: A9,A12


expr_map = {
    # TODO , 'C6' was transformed later, but not yet tested
    **dict.fromkeys(['A2', 'A4', 'B5', 'B6', 'B7', 'B9', 'B11', 'C5'], 'none'), 
    **dict.fromkeys(['A1','A3', 'A6', 'A8', 'B1','B4', 'B10','C3'], 'low'),
    **dict.fromkeys(['A7', 'A10','B12','C1','F4'], 'medium'),
    **dict.fromkeys(['A11', 'B2', 'B3','A5'], 'high'),
    **dict.fromkeys(['A9', 'A12', 'B8'], 'super_high')
}
df['expression_levels'] = df["Well Position"].map(expr_map)

# populate sds_page_size_appearance for every visible band?
# TODO

# populate large_scale_expressed for A12,A9,B8,A1,B12,F4
scaled_expressed_ids = ['A12','A9','B8','A1','B12','F4',"A11","B3","B10"]
df['large_scale_expressed'] = df["Well Position"].isin(scaled_expressed_ids)

# populate scaled_purified for A12,A9,B8 TODO add ,'A1','B12','F4'
# scaled_purified_ids = ['A12','A9','B8'] \

# Note: A9,B8 had some aggregation (small pellet after spining down). A12 did not

A280_after_NI_NTA_IMAC = {
    "A12": 13.41,
    "A9": 6.2,
    "B8": 16.11,
    #"A1" Not Measured, 
    #"B12" Not Measured, 
    #"F4": Not Measured, 
    "A11":6.565,
    "B3":9.067,
    "B10":5.780,
}


df['A280_after_NI_NTA_IMAC'] = df["Well Position"].map(A280_after_NI_NTA_IMAC)
volume_after_NI_NTA_IMAC = 3.2
# calculate molar conc asuming sample is entirely my protein df.sequence_molar_extinction_280

df['molar_conc_after_NI_NTA_IMAC'] = df['A280_after_NI_NTA_IMAC'] / df['sequence_molar_extinction_280_w_prefix']
df['mg_per_ml_after_NI_NTA_IMAC'] = df['molar_conc_after_NI_NTA_IMAC'] * df['mass_w_prefix']
df['mg_after_NI_NTA_IMAC'] = df['mg_per_ml_after_NI_NTA_IMAC'] * volume_after_NI_NTA_IMAC

# after SEC (central fractions)
A280_after_SEC = {
    "A12": 4.93,
    "A9": 2.18,
    "B8": 6.758,
    "A1": 0.820,
    "B12": 2.510,
    "F4": 2.427, 
    "A11":2.478,
    "B3":2.939,
    "B10":2.556,
}
volume_after_sec = {
    "A12": 5.1,
    "A9": 5.1,
    "B8": 3.4,
    "A1": 3.4,
    "B12": 5.1,
    "F4": 5.1,
    "A11": 5.1,
    "B3":3.4,
    "B10":3.4,
}
df["volume_after_sec"] = df["Well Position"].map(volume_after_sec)

df['A280_after_SEC'] = df["Well Position"].map(A280_after_SEC)
df['molar_conc_after_SEC'] = df['A280_after_SEC'] / df['sequence_molar_extinction_280_w_prefix']
df['mg_per_ml_after_SEC'] = df['molar_conc_after_SEC'] * df['mass_w_prefix']
df['mg_after_SEC'] = df['mg_per_ml_after_SEC'] * df['volume_after_sec']


# Biophysical characterisation info (CD and biophysical SEC used same sample. Was assumed it is 20 uM, however it was a bit less.)
cd_sample_A280 = {
    "A12": 0.395, # later measure: 0.406
    "A9": 0.168, # later measure: 0.190
    "B8": 0.517, # later measure: 0.537
    "A1":0.173,
    "B12": 0.210,
    "F4": 0.211,
    "A11": 0.569,
    "B3":0.764,
    "B10":0.175,
}

df["cd_sample_A280"] = df["Well Position"].map(cd_sample_A280)
df["cd_sample_molar_conc"] = df["cd_sample_A280"] / df["sequence_molar_extinction_280_w_prefix"]
df["cd_path_length_mm"]=1

# crystal screens set up with 3+3 uL

    # # plate 1, position 2:
    # "B12": 19.19,

    # # plate 2, position 1:
    # "A12": 36.56,
    # # plate 2, position 2:
    # "B8": 43.35,

    # # plate 3, position 1:
    # "A1": 13.82,
    # # plate 3, position 2:
    # "B3": 37.62,
crystal_screens_info = {
    # plate 1, position 1:
    "A9": {
        "plate": 1,
        "position": 1,
        "date": "2025-04-11",
        "A280": 19.40
    },
    "B12":{
        "plate": 1,
        "position": 2,
        "date": "2025-04-11",
        "A280": 19.19
    },
    "A12":{
        "plate": 2,
        "position": 1,
        "date": "2025-04-14",
        "A280": 36.56
    },
    "B8":{
        "plate": 2,
        "position": 2,
        "date": "2025-04-14",
        "A280": 43.35
    },
    "A1":{
        "plate": 3,
        "position": 1,
        "date": "2025-04-17",
        "A280": 13.82
    },
    "B3":{
        "plate": 3,
        "position": 2,
        "date": "2025-04-17",
        "A280": 37.62
    }
}
# unpack all of these columns to df

df["crystal_screens_info"] = df["Well Position"].map(crystal_screens_info)
# df["mg_per_ml_crystal_screens"] = df["crystal_screens_info"].map(lambda x: x["mg_per_ml"] if x else np.nan)
# df["plate_crystal_screens"] = df["crystal_screens_info"].map(lambda x: x["plate"] if x else np.nan)
# df["position_crystal_screens"] = df["crystal_screens_info"].map(lambda x: x["position"] if x else np.nan)
# df["date_crystal_screens"] = df["crystal_screens_info"].map(lambda x: x["date"] if x else np.nan)
    


In [ ]:
df[useful_cols].to_csv("/home/tadas/code/deltaproteinsBristol/experimental_results/deltaprot_designs_data_with_results.csv", index=False)
df.to_pickle("/home/tadas/code/deltaproteinsBristol/deltaprot_designs_all_data_with_results.pkl")

In [ ]:
df[df['large_scale_expressed']][["Well Position","Name","mass_w_prefix","expression_levels","mg_per_ml_after_SEC","mg_after_SEC","molar_conc_after_SEC","cd_sample_molar_conc"]]

In [ ]:
# set limit to column width
pd.set_option('display.max_colwidth', 1)

In [ ]:
df[["Well Position","cd_sample_molar_conc","Name","expression_levels"]]

In [ ]:
df[["Well Position","cd_sample_molar_conc","Name","A280_after_SEC","volume_after_sec","sequence_molar_extinction_280_w_prefix","molar_conc_after_SEC","mg_per_ml_after_SEC","mg_after_SEC"]]

In [ ]:
df[["Well Position","Name","mass","mass_w_prefix","charge","A280_after_SEC","volume_after_sec","sequence_molar_extinction_280_w_prefix","molar_conc_after_NI_NTA_IMAC","molar_conc_after_SEC","mg_per_ml_after_SEC","mg_after_SEC"]]

In [ ]:
# mass correct: A9, A11, B2, C1, F4, B12
# mass questionable: A12 (has 2 bands), B3 (2 bands)
# mass wrong: 

In [ ]:
# unlimit pandas column display number
pd.set_option('display.max_columns', None)

In [ ]:
df[df["expression_levels"].isna()][["Well Position","Name","mass_w_prefix","expression_levels","large_scale_expressed"]]

In [ ]:
df["expression_levels"]

In [ ]:
df[df["expression_levels"].isin(["low","medium","high","super_high"])].sort_values(["large_scale_expressed","expression_levels"],ascending=[False,True])[["Well Position","Name","mass_w_prefix","expression_levels","large_scale_expressed"]]

In [ ]:
x=df[df["expression_levels"].isin(["high","super_high"])]
x

In [ ]:
# Choices for large scale expression

# A12	no_disulfide_b5ininn super_high
# A9	no_disulfide_b4nnnny
# B8	no_disulfide_h6i_i_i

# A1 no_disulfide_b3iii (low-medium)
# B12	no_disulfide_l5iiin (medium)
# F4	variable_linkers_l6innni	(medium)


In [ ]:
p